<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/06.GraphWriter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Partisan-Responses-master')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


Install all requirements.

In [ ]:
!pip install neuralcoref allennlp hnswlib allennlp-models torch==1.5.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -U spacy[cuda101]==2.1.1
!python -m spacy download en
!pip install wandb

     |████████████████████████████████| 4.5MB 8.9MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Requirement already up-to-date: spacy[cuda101]==2.1.1 in /usr/local/lib/python3.6/dist-packages (2.1.1)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
!pip uninstall thinc
!pip install thinc==7.0.3

Uninstalling thinc-7.0.8:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/thinc-7.0.8.dist-info/*
    /usr/local/lib/python3.6/dist-packages/thinc/*
Proceed (y/n)? y
  Successfully uninstalled thinc-7.0.8
     |████████████████████████████████| 2.1MB 9.3MB/s 
     |████████████████████████████████| 491kB 42.3MB/s 
  Created wheel for thinc-gpu-ops: filename=thinc_gpu_ops-0.0.4-cp36-cp36m-linux_x86_64.whl size=221833 sha256=93af19c0e01c1f83c4781aaf452872d2e286a09c89320a8d65b285e2e2ea11af
  Stored in directory: /root/.cache/pip/wheels/eb/ba/a3/9af9f326ed0d75a4540378af64a05a0e42be39d9b8513f3aea
Successfully built thinc-gpu-ops


In [ ]:
!pip uninstall torchtext
!pip install torchtext==0.6.0

Uninstalling torchtext-0.7.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/torchtext-0.7.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torchtext/*
Proceed (y/n)? y
  Successfully uninstalled torchtext-0.7.0
     |████████████████████████████████| 71kB 4.7MB/s 


Import everything

In [ ]:
import neuralcoref
import spacy
import re
import pickle
import pandas as pd
import gc
from Answer import Answer
import utils

100%|██████████| 40155833/40155833 [00:02<00:00, 19570926.72B/s]
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


100%|██████████| 1345947288/1345947288 [00:22<00:00, 58826279.53B/s]


Setup everything for coref

In [ ]:
nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)
disabled =  ["ner"]

identifier='coref_speeches'
last_check=136
step_size=1000

def load_pickles(identifier, checkpoint):
  if os.path.exists("df"+identifier+str(checkpoint)+".pickle"):
    df = pd.read_pickle("df"+identifier+str(checkpoint)+".pickle")
    return df

def dump_pickles(identifier, checkpoint, df):
  df.to_pickle("df"+identifier+str(checkpoint)+".pickle")
  gc.collect()
  return 

def make_corefs(content):
    content=str(re.sub("\.(?=\s[a-z0-9]|\sI[\W\s])", ",", content))
    doc=nlp(content, disable=disabled)
    return doc._.coref_resolved

def mask_entity(speech): 
  ents = nlp(speech, disable=[['tagger', 'parser']]).ents
  to_mask = filter(lambda ent: ent.label_ == "PERSON" or ent.label_ == "TIME", ents)
  #[ent.start_char, ent.end_char for ent in ents]
  for ent in to_mask:
      speech = speech[:ent.start_char] + ent.label_ + speech[ent.end_char:]
  return speech

In [ ]:
df=load_pickles(identifier, last_check)
#df = pd.read_pickle("all_speech_sentence_filtered.pkl")
#df = df.sample(n=50000, random_state=36)

In [ ]:
i = 0
step_size = 5
[make_corefs(df.iloc[step_size*(i-1)+j]['Questions']) for j in range(step_size)]

['the modification is not in the body of the amendment but in the stated purpose. To make the statement of purpose acceptable to the other side, we have stricken the word "emergency." That meets with the approval of the other side, so it is now as changed. As we all know, the budget resolution before we all includes room in the budget in 2008 and 2009 for an additional stimulus package. The distinguished chairman included this "insurance policy" against further economic downturn, and I commend The distinguished chairman for this "insurance policy" against further economic downturn, I also thank The distinguished chairman and Senator PERSON and all the distinguished bipartisan cosponsors for working with The distinguished chairman to adopt the amendment. the amendment simply allocates more of the stimulus money for "ready to go" infrastructure projects. the amendment moves $3.5 billion from the allowances functions to the transportation function and designates the amendment as fiscal ye

Do coref

In [ ]:
import time
for i in range(last_check+1,int(df.shape[0]/step_size)):
  print(i)
  df['Questions'][step_size*(i-1):step_size*(i-1)+step_size] = [make_corefs(df.iloc[step_size*(i-1)+j]['Questions']) for j in range(step_size)]
  dump_pickles(identifier,i,df)
print('done with ordered')
for j in range(df.shape[0]-int(df.shape[0]/step_size)*step_size):
  df.iloc[step_size*int(df.shape[0]/step_size)+j]['Questions']=make_corefs(df.iloc[step_size*int(df.shape[0]/step_size)+j]['Questions'])
df.to_pickle(identifier+".pkl")

done with ordered


In [ ]:
# NER to mask person and time
spacy.prefer_gpu()
last_check = 0
identifier = "ner_speeches"
nlp = spacy.load('en')

for i in range(last_check+1,int(df.shape[0]/step_size)):
  print(i)
  df['Questions'][step_size*(i-1):step_size*(i-1)+step_size] = [mask_entity(df.iloc[step_size*(i-1)+j]['Questions']) for j in range(step_size)]
  dump_pickles(identifier,i,df)
print('done with ordered')
for j in range(df.shape[0]-int(df.shape[0]/step_size)*step_size):
  df.iloc[step_size*int(df.shape[0]/step_size)+j]['Questions']=make_corefs(df.iloc[step_size*int(df.shape[0]/step_size)+j]['Questions'])
df.to_pickle(identifier+".pkl")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
done with ordered


In [ ]:
last_ckpt = 0
df = pd.read_pickle("ner_speeches.pkl")
df.shape

FileNotFoundError: ignored

In [ ]:
import time
import pandas as pd
# import allennlp_models.structured_prediction
# import allennlp_models.coref
import nltk
import re
import utils
import hnswlib

auxillary_verbs=['can','could','may','might','must','shall','should','will','would'] #https://englishstudyonline.org/auxiliary-verbs/
distance_threshold=0.5

class Answer:
    def __init__(self, answer):
        self.content = answer
        self.phrase_corpus=[]
        self.phrase_index=None
        self.triplet=[]
        self.triplet_id=[]
        self.parsed=[]
        
    def change_comma(self):
        """
        Replace improper period to comma
        """
        self.content = re.sub("\.(?=\s[a-z0-9]|\sI[\W\s])", ",", self.content)
        
    def create_triplet(self):
        """
        Generate (subject, verb, object) triplets of a speech text
        Param:
        ========
        coref_extractor: allennlp coreferece resolution predictor
        oi_extractor: allennlp open information extractor
        Return:
        ========
        triplets: list, a list of triplet tuples except the last item being party string
        """
        oie_result=self.create_oieresult()
        triplets = self._find_triplets(oie_result)
        triplets.append(self.party)
        return triplets
    
    def create_coref(self):
        return utils.coref_extractor.coref_resolved(self.content)
        
    def create_oieresult(self):
        coref_content=self.content
        sents = nltk.tokenize.sent_tokenize(coref_content)
        sents = [{"sentence":s} for s in sents] #Format for oie batch predictor
        oie_result = utils.open_info_extractor.predict_batch_json(sents)
        oie_result = [i['verbs'] for i in oie_result]
        return oie_result
    
    def add_phrase(self, phrase):
        Id=len(self.phrase_corpus)
        self.phrase_corpus.append(phrase)
        self.phrase_index = hnswlib.Index('cosine', 512)
        self.phrase_index.init_index(len(self.phrase_corpus), ef_construction=200, M=48, random_seed=36)
        if len(self.phrase_corpus) > 1:
            self.phrase_index.load_index("phrase_index", max_elements=len(self.phrase_corpus))
        self.phrase_index.add_items(utils.model([phrase]))
        self.phrase_index.save_index("phrase_index")
        return Id, phrase

    def deduplicate(self,phrase):
        if len(self.phrase_corpus)==0:
            return self.add_phrase(phrase)
        nearest_neighbor=self.phrase_index.knn_query(utils.model([phrase]))
        if nearest_neighbor != []:
            closest_neighbor, closest_distance = nearest_neighbor
        if closest_neighbor[0] == []:
            return self.add_phrase(phrase)
        if closest_distance[0][0] > distance_threshold:
            return self.add_phrase(phrase)
        return_phrase=self.phrase_corpus[closest_neighbor[0][0]]
        return self.phrase_corpus.index(return_phrase),return_phrase

    def create_training(self,verb_dict, verb_list):
        self.change_comma()
        triplets = self.create_oieresult()
        return_text=""
        
        for sentence in triplets:
            if len(sentence)==0:
                self.parsed.append('str(len(self.phrase_corpus))+" -1"')
                continue
            text=re.sub('\[[^\s]*','',sentence[0]['description'])
            text=re.sub('\]','',text).split()
            tags=[False]*len(sentence[0]['tags'])
            for triplet in sentence:
                arg_points=[x in ['I-ARG0','B-ARG0','I-ARG1','B-ARG1'] for x in triplet['tags']]
                abort=False
                for others in sentence:
                    for place in range(len(others['tags'])):
                        if others['tags'][place][-2:]=='-V' and arg_points[place]:
                            abort=True
                            break
                        if abort:
                            break
                if abort:
                    continue
                subject=' '.join([text[x] for x in range(len(text)) if triplet['tags'][x] in ['I-ARG0','B-ARG0']])
                objekt=' '.join([text[x] for x in range(len(text)) if triplet['tags'][x] in ['I-ARG1','B-ARG1']])
                verb=triplet['verb']
                verb=utils.lemmatize(triplet['verb'])
                if verb in auxillary_verbs:
                    continue
                if len(subject)==0:
                    continue
                if len(objekt)==0:
                    continue
                if verb in verb_dict.keys():
                    verb_id=verb_dict[verb]
                else:
                    verb_id=len(verb_list)
                    verb_list.append(verb)
                    verb_dict[verb]=verb_id

                max_id=len(self.phrase_corpus)
                subject_id,subject=self.deduplicate(subject)
                if subject_id==max_id:
                    self.parsed.append(str(subject_id))
                tags=[str(subject_id) if triplet['tags'][x] in ['I-ARG0','B-ARG0'] else tags[x] for x in range(len(text))]

                max_id=len(self.phrase_corpus)
                objekt_id,objekt=self.deduplicate(objekt)
                if objekt_id==max_id:
                    self.parsed.append(str(objekt_id))
                tags=[str(objekt_id) if triplet['tags'][x] in ['I-ARG1','B-ARG1'] else tags[x] for x in range(len(text))]
                if (subject,objekt,verb) not in self.triplet:
                    self.triplet.append((subject,objekt,verb))
                    self.triplet_id.append((subject_id,verb_id,objekt_id))
                    self.parsed.append("str(len(self.phrase_corpus)+"+str(len(self.triplet))+")")
            self.parsed.append('str(len(self.phrase_corpus))+" -1"')
            text=['<phrase_'+str(tags[x])+'>' if tags[x] else text[x] for x in range(len(text))]
            text.append(None)
            text=[text[x] for x in range(len(text)-1) if (text[x]!=text[x+1] or text[x][0]!='<')]
            return_text=return_text+' '+' '.join(text)
        
        return self.phrase_corpus,self.triplet_id,return_text[1:],[eval(x, {"self": self}) for x in self.parsed]

    def create_test(self,verb_dict, verb_list):
        self.change_comma()
        triplets = self.create_oieresult()
        return_text=""
        for sentence in triplets:
            if len(sentence)==0:
                self.parsed.append('str(len(self.phrase_corpus))+" -1"')
                continue
            text=re.sub('\[[^\s]*','',sentence[0]['description'])
            text=re.sub('\]','',text).split()
            tags=[False]*len(sentence[0]['tags'])
            for triplet in sentence:
                arg_points=[x in ['I-ARG0','B-ARG0','I-ARG1','B-ARG1'] for x in triplet['tags']]
                abort=False
                for others in sentence:
                    for place in range(len(others['tags'])):
                        if others['tags'][place][-2:]=='-V' and arg_points[place]:
                            abort=True
                            break
                        if abort:
                            break
                if abort:
                    continue
                subject=' '.join([text[x] for x in range(len(text)) if triplet['tags'][x] in ['I-ARG0','B-ARG0']])
                objekt=' '.join([text[x] for x in range(len(text)) if triplet['tags'][x] in ['I-ARG1','B-ARG1']])
                verb=triplet['verb']
                verb=utils.lemmatize(triplet['verb'])
                if verb in auxillary_verbs:
                    continue
                if len(subject)==0:
                    continue
                if len(objekt)==0:
                    continue
                verb = verb.upper()
                if verb not in verb_dict.keys():
                    print(verb)
                    continue #if verb does not exist in verb_dict it can not be used to create
                verb_id=verb_dict[verb]
                max_id=len(self.phrase_corpus)
                subject_id,subject=self.deduplicate(subject)
                if subject_id==max_id:
                    self.parsed.append(str(subject_id))
                tags=[str(subject_id) if triplet['tags'][x] in ['I-ARG0','B-ARG0'] else tags[x] for x in range(len(text))]

                max_id=len(self.phrase_corpus)
                objekt_id,objekt=self.deduplicate(objekt)
                if objekt_id==max_id:
                    self.parsed.append(str(objekt_id))
                tags=[str(objekt_id) if triplet['tags'][x] in ['I-ARG1','B-ARG1'] else tags[x] for x in range(len(text))]
                if (subject,objekt,verb) not in self.triplet:
                    self.triplet.append((subject,objekt,verb))
                    self.triplet_id.append((subject_id,verb_id,objekt_id))
                    self.parsed.append("str(len(self.phrase_corpus)+"+str(len(self.triplet))+")")
            self.parsed.append('str(len(self.phrase_corpus))+" -1"')
            text=['<phrase_'+str(tags[x])+'>' if tags[x] else text[x] for x in range(len(text))]
            text.append(None)
            text=[text[x] for x in range(len(text)-1) if (text[x]!=text[x+1] or text[x][0]!='<')]
            return_text=return_text+' '+' '.join(text)
        return self.phrase_corpus,self.triplet_id,return_text[1:],[eval(x, {"self": self}) for x in self.parsed]


Setup for parsing

In [ ]:
identifier="ner_speeches"
last_check=215
step_size=100
#df is still from the cell above above

def parse_entry(entry,verb_dict,verb_list):
    result=dict()
    #result['question']=' '.join([token.text for token in utils.sp(entry['question'])])
    result['question']=' '
    phrase_corpus, triplet_id, parsed_text,parsed=Answer(entry['Questions']).create_training(verb_dict,verb_list)
    result['corpus']=' ; '.join(phrase_corpus)
    result['tags']=' '.join(['<phrase>']*len(phrase_corpus))
    result['triplet_id']=' ; '.join([re.sub('\,','',str(x))[1:-1] for x in triplet_id])
    result['parsed_text']=parsed_text
    result['parsed']=' '.join([str(x) for x in parsed])
    return result

def load_pickles(identifier, checkpoint):
  if os.path.exists("verb_dict"+identifier+str(checkpoint)+".pickle"):
    with open('verb_dict'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      verb_dict = pickle.load(handle)
    with open('verb_list'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      verb_list = pickle.load(handle)
    with open('result'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      result = pickle.load(handle)
  else:
    print('File does not exist yet.')
    verb_dict=dict()
    verb_list=[]
    result=[]
  return verb_dict, verb_list, result

def dump_pickles(identifier, checkpoint, verb_dict, verb_list, result):
  with open("verb_dict"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(verb_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open("verb_list"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(verb_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open("result"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)
  return 

verb_dict, verb_list, result=load_pickles(identifier, last_check)

Process data

In [ ]:
for i in range(last_check+1,int(df.shape[0]/step_size)):
  print(i)
  df[step_size*(i-1):step_size*i].apply(lambda x: result.append(parse_entry(x,verb_dict,verb_list)), axis=1)
  dump_pickles(identifier,i,verb_dict,verb_list,result)
print('done with ordered')
df[step_size*i:].apply(lambda x: result.append(parse_entry(x,verb_dict,verb_list)), axis=1)
dump_pickles(identifier,1+df.shape[0]/step_size,verb_dict,verb_list,result)

137


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216


Post-processing of data

In [ ]:
df=pd.DataFrame(result) 
df=df.sample(frac=1, random_state=36)
df=df.dropna(axis=0, how='any')
print(df.shape)
df=df[df.iloc[:,2].apply(lambda x: len(x)>1)]
print(df.shape)
df=df[df.iloc[:,4].apply(lambda x: len(x)<2000)]
print(df.shape)
df=df[df.iloc[:,1].apply(lambda x: len(x)<2000)]
print(df.shape)
df.head()

(21400, 6)
(21223, 6)
(10027, 6)
(10027, 6)


,question,corpus,tags,triplet_id,parsed_text,parsed
12192,,I ; the chairman ; a provision ; the Journey T...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 35 1 ; 2 350 3 ; 3 2253 3 ; 4 52 5 ; 0 307 6...,first I want to thank the ranking member . PER...,10 -1 10 -1 10 -1 0 1 11 10 -1 10 -1 10 -1 2 3...
10401,,we ; terrorism ; the Federal Government ; many...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 365 1 ; 2 39 3 ; 0 256 4 ; 5 89 6 ; 0 47 7 ;...,we all want to do what <phrase_0> all should t...,0 1 13 2 3 14 12 -1 4 15 12 -1 5 6 16 12 -1 12...
10036,,I ; a very sound request to the contrary on ; ...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 479 1 ; 0 3624 2 ; 3 39 4 ; 5 146 6 ; 5 414 ...,"this is a healthy discussion , a healthy debat...",0 1 11 10 -1 2 12 3 4 13 10 -1 5 6 14 10 -1 7 ...
12446,,last act ; PERSON and devotion Sergeant Herrer...,<phrase> <phrase> <phrase> <phrase> <phrase>,0 3 1 ; 1 481 2 ; 3 268 1 ; 4 30 4,I rise today to commemorate one of Utahs falle...,5 -1 5 -1 5 -1 5 -1 5 -1 0 1 6 5 -1 2 7 3 8 4 ...
681,,you ; with a $ 4.7 trillion deficit ; us ; bad...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 115 1 ; 2 89 3 ; 4 1360 5 ; 2 165 6 ; 0 355 ...,It is between 375 and 400 . But when <phrase_0...,16 -1 16 -1 0 1 17 16 -1 16 -1 16 -1 2 3 18 4 ...


Write to variables

In [ ]:
identifier='../GraphWriter-master/data/preprocessed'
m=df.iloc[:50]
m.to_csv(identifier+'.control.tsv', sep='\t', index=False, header=False)
m=df.iloc[50:1050]
m.to_csv(identifier+'.val.tsv', sep='\t', index=False, header=False)
m=df.iloc[1050:2050]
m.to_csv(identifier+'.test.tsv', sep='\t', index=False, header=False)
m=df.iloc[2050:]
m.to_csv(identifier+'.train.tsv', sep='\t', index=False, header=False)

with open(identifier+'.vocab', 'w') as filehandle:
    filehandle.writelines("%s\n" % verb.upper() for verb in verb_list)

Train with GraphWriter

In [47]:
os.chdir('/content/drive/My Drive/GraphWriter-master')
!python ../GraphWriter-master/train.py -save ../GraphWriter-master/full_speeches

2020-08-23 08:28:46.451857: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Save File Exists, OverWrite? <CTL-C> for noy
Loading Data from  data/preprocessed.train.tsv
building vocab
Sorting training data by len
ds sizes:	1125	3790	3062	1000	Vocab sizes:
src 4
ent 19591
nerd 4
rel 17893
out 11284
graph
cuda:0
epoch  0 lr 0.1
Training	1
2
2709 of like 40k -- current avg loss  6.598582795820099
4309 of like 40k -- current avg loss  6.269156448911365
3
5707 of like 40k -- current avg loss  6.093150338546729
6507 of like 40k -- current avg loss  6.009009810770567
7307 of like 40k -- current avg loss  5.931816013368027
AVG TRAIN LOSS:  5.875482077764152	 PPL:  356.1963336197094
Evaluating	I amendment to the to the gentleman of the that the to . be be able . a to <unk> of <unk> to . the <unk> . . . . of to the the . . 

Use file with lowest vloss in the folder "full_speeches" of GraphWriter-master to generate responses on test data.


E.g. in the next cell "9.vloss-3.980539.lr-0.1" was used.

In [50]:
!python ../GraphWriter-master/generator.py -save=../GraphWriter-master/full_speeches/12.vloss-4.299755.lr-0.1

2020-08-23 12:22:17.465426: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Loading Data from  data/preprocessed.train.tsv
building vocab
Vocab sizes:
src 4
ent 19591
nerd 4
rel 17893
out 11284
graph
0 1000
/content/drive/My Drive/GraphWriter-master/models/newmodel.py:153: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cx = torch.tensor(hx)
I rise in opposition to the Thomas substitute , as IT does not represent REAL REFORM , OUR CAMPAIGN SYSTEM is broken and needs REAL REFORM and not <unk> and <unk> AROUNDTHEEDGE SOLUTIONS offered BY THE HONORABLE CHAIRMAN , THE GENTLEMAN FROM CALIFORNIA . Although THE THOMAS SUBSTITUTE contains SOME IMPORTANT REFORMS OF THE FEDER